In [1]:
!pip install --upgrade kiteconnect
!pip install python-dotenv
!pip install python-dateutil

Requirement already up-to-date: kiteconnect in c:\users\nisha\miniconda3\lib\site-packages (3.9.4)


In [1]:
import logging
import os
from kiteconnect import KiteConnect
from dotenv import load_dotenv
load_dotenv()
import bs_threading
import importlib
import time
from datetime import datetime, timedelta
import dateutil
import pandas as pd
importlib.reload(bs_threading)
from bs_threading import bs_threadify, bs_make_throttle_ready_func


In [2]:
KITE_CONNECT_API_KEY = os.getenv("KITE_CONNECT_API_KEY")
KITE_CONNECT_API_SECRET = os.getenv("KITE_CONNECT_API_SECRET")

In [3]:

logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key=KITE_CONNECT_API_KEY)



### Get the url based on your api key, which will have request token, this is in order to verify the app itself(bsstonks) is valid and it'll get redirected to the callback url

In [4]:
print(kite.login_url())

https://kite.trade/connect/login?api_key=7e6nr4sfi674uleu&v=3


### Given we have request token and redirected to our callback with it, our app is valid, but is the app in the hands/server of the right person or not? only API secret can verify that. 
### We'll pass that api secret with the request token to create a session which give us Access_token, to the rightful person that we are

In [5]:
# Redirect the user to the login url obtained
# from kite.login_url(), and receive the request_token
# from the registered redirect url after the login flow.
# Once you have the request_token, obtain the access_token
# as follows.

kite_session_data = kite.generate_session("IHtXzJoMdoOEf2KywACeNKwh7VBBrLJM", api_secret=KITE_CONNECT_API_SECRET)
print("THE TOKEN:",kite_session_data["access_token"])
kite.set_access_token(kite_session_data["access_token"])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "POST /session/token HTTP/1.1" 200 None


THE TOKEN: i46t3o7RJEb2VOWW4jMpFSMFE1ELBKJh


In [4]:
kite.set_access_token("i46t3o7RJEb2VOWW4jMpFSMFE1ELBKJh")

In [5]:
kite.orders()
kite.holdings()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /orders HTTP/1.1" 200 30
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/holdings HTTP/1.1" 200 None


[{'tradingsymbol': 'BHEL',
  'exchange': 'BSE',
  'instrument_token': 128026372,
  'isin': 'INE257A01026',
  'product': 'CNC',
  'price': 0,
  'quantity': 250,
  'used_quantity': 0,
  't1_quantity': 0,
  'realised_quantity': 250,
  'authorised_quantity': 0,
  'authorised_date': '2021-07-10 00:00:00',
  'opening_quantity': 250,
  'collateral_quantity': 0,
  'collateral_type': '',
  'discrepancy': False,
  'average_price': 67.9,
  'last_price': 66.45,
  'close_price': 66.45,
  'pnl': -362.5000000000007,
  'day_change': 0,
  'day_change_percentage': 0},
 {'tradingsymbol': 'MANGALAM',
  'exchange': 'BSE',
  'instrument_token': 136355076,
  'isin': 'INE584F01014',
  'product': 'CNC',
  'price': 0,
  'quantity': 1,
  'used_quantity': 0,
  't1_quantity': 0,
  'realised_quantity': 1,
  'authorised_quantity': 0,
  'authorised_date': '2021-07-10 00:00:00',
  'opening_quantity': 1,
  'collateral_quantity': 0,
  'collateral_type': '',
  'discrepancy': False,
  'average_price': 132.85,
  'last_pric

In [6]:
all_instruments = kite.instruments()
all_instruments_nse = kite.instruments(exchange="NSE")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments HTTP/1.1" 200 1100278
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NSE HTTP/1.1" 200 60569


In [7]:
df_all_instruments = pd.DataFrame(all_instruments)
os.makedirs("input/kite_instruments", exist_ok=True)
df_all_instruments.to_csv("input/kite_instruments/instrument_list.csv", index=False)
df_all_instruments

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,556726790,2174714,EURINR21AUG84.5000CE,EURINR,0.0,2021-08-27,84.50,0.0025,1,CE,BCD-OPT,BCD
1,556722694,2174698,EURINR21AUG84.5000PE,EURINR,0.0,2021-08-27,84.50,0.0025,1,PE,BCD-OPT,BCD
2,556833798,2175132,EURINR21AUG84.7500CE,EURINR,0.0,2021-08-27,84.75,0.0025,1,CE,BCD-OPT,BCD
3,556829958,2175117,EURINR21AUG84.7500PE,EURINR,0.0,2021-08-27,84.75,0.0025,1,PE,BCD-OPT,BCD
4,556823046,2175090,EURINR21AUG85.0000CE,EURINR,0.0,2021-08-27,85.00,0.0025,1,CE,BCD-OPT,BCD
...,...,...,...,...,...,...,...,...,...,...,...,...
83227,1945089,7598,ZODJRDMKJ-BE,ZODIAC JRD MKJ-,0.0,,0.00,0.0500,1,EQ,NSE,NSE
83228,2916865,11394,ZOTA,ZOTA HEALTH CARE,0.0,,0.00,0.0500,1,EQ,NSE,NSE
83229,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0.0,,0.00,0.0500,1,EQ,NSE,NSE
83230,979713,3827,ZUARIGLOB,ZUARI GLOBAL,0.0,,0.00,0.0500,1,EQ,NSE,NSE


In [22]:
print("SEgemtns:",df_all_instruments["segment"].unique())
print("instrument_type:",df_all_instruments["instrument_type"].unique())
print("exchange:",df_all_instruments["exchange"].unique())
print("exchange:",df_all_instruments[df_all_instruments["exchange"]=="NFO"]['instrument_type'].unique())

derivatives_df = df_all_instruments[df_all_instruments["exchange"]=="NFO"]
derivatives_df

options_df = df_all_instruments[df_all_instruments["segment"]=="NFO-OPT"]
options_df

reliance_options_df = options_df[options_df["tradingsymbol"].str.lower().str.contains("relia") & options_df["tradingsymbol"].str.lower().str.contains("2400")]
reliance_options_df

SEgemtns: ['BCD-OPT' 'BCD-FUT' 'BCD' 'BSE' 'INDICES' 'CDS-OPT' 'CDS-FUT' 'MCX-FUT'
 'MCX-OPT' 'NFO-OPT' 'NFO-FUT' 'NSE']
instrument_type: ['CE' 'PE' 'FUT' 'EQ']
exchange: ['BCD' 'BSE' 'MCX' 'NSE' 'CDS' 'NFO']
exchange: ['CE' 'PE' 'FUT']


,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
70934,27437570,107178,RELIANCE21JUL2400CE,RELIANCE,0.0,2021-07-29,2400.0,0.05,250,CE,NFO-OPT,NFO
70935,27437826,107179,RELIANCE21JUL2400PE,RELIANCE,0.0,2021-07-29,2400.0,0.05,250,PE,NFO-OPT,NFO
71068,28513282,111380,RELIANCE21AUG2400CE,RELIANCE,0.0,2021-08-26,2400.0,0.05,250,CE,NFO-OPT,NFO
71069,28513538,111381,RELIANCE21AUG2400PE,RELIANCE,0.0,2021-08-26,2400.0,0.05,250,PE,NFO-OPT,NFO
71202,30570754,119417,RELIANCE21SEP2400CE,RELIANCE,0.0,2021-09-30,2400.0,0.05,250,CE,NFO-OPT,NFO
71203,30571010,119418,RELIANCE21SEP2400PE,RELIANCE,0.0,2021-09-30,2400.0,0.05,250,PE,NFO-OPT,NFO


In [23]:
end = datetime.now()
end = end - timedelta(days=1)
start = end - timedelta(days=260)
kite.historical_data(27437570, start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"), "day", oi=True, continuous=True)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/27437570/day?from=2020-10-22&to=2021-07-09&interval=day&continuous=1&oi=1 HTTP/1.1" 200 3420


[{'date': datetime.datetime(2020, 10, 22, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 4.5,
  'high': 4.7,
  'low': 2.75,
  'close': 3.05,
  'volume': 1891730,
  'oi': 2865875},
 {'date': datetime.datetime(2020, 10, 23, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 2.95,
  'high': 2.95,
  'low': 1.35,
  'close': 1.55,
  'volume': 1675590,
  'oi': 2553280},
 {'date': datetime.datetime(2020, 10, 26, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 1.2,
  'high': 1.2,
  'low': 0.75,
  'close': 1,
  'volume': 1454400,
  'oi': 2075045},
 {'date': datetime.datetime(2020, 10, 27, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 0.95,
  'high': 1.1,
  'low': 0.5,
  'close': 0.6,
  'volume': 744370,
  'oi': 1841735},
 {'date': datetime.datetime(2020, 10, 28, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 0.25,
  'high': 0.55,
  'low': 0.1,
  'close': 0.2,
  'volume': 475205,
  'oi': 1525605},
 {'date': datetime.datetime(2020, 10, 29, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 0.2,
  'high': 

In [24]:
end = datetime.now()
end = end - timedelta(days=1)
start = end - timedelta(days=260)
kite.historical_data(28513282, start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d"), "day", oi=True, continuous=True)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/28513282/day?from=2020-10-22&to=2021-07-09&interval=day&continuous=1&oi=1 HTTP/1.1" 200 3381


[{'date': datetime.datetime(2020, 10, 22, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 26.4,
  'high': 29,
  'low': 23.5,
  'close': 26.2,
  'volume': 299465,
  'oi': 468135},
 {'date': datetime.datetime(2020, 10, 23, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 26.65,
  'high': 29.45,
  'low': 22.75,
  'close': 23.3,
  'volume': 486820,
  'oi': 639835},
 {'date': datetime.datetime(2020, 10, 26, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 21.65,
  'high': 21.65,
  'low': 13,
  'close': 14.3,
  'volume': 929200,
  'oi': 861025},
 {'date': datetime.datetime(2020, 10, 27, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 12.9,
  'high': 16.25,
  'low': 11,
  'close': 13.7,
  'volume': 740330,
  'oi': 1010000},
 {'date': datetime.datetime(2020, 10, 28, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 14.9,
  'high': 16.2,
  'low': 12.45,
  'close': 13.2,
  'volume': 596405,
  'oi': 1132210},
 {'date': datetime.datetime(2020, 10, 29, 0, 0, tzinfo=tzoffset(None, 19800)),
  'open': 12.5,
  

In [13]:
for x in all_data:
    x['date'] = x.strftime("%Y-%m-%d")
import json
json.dumps(all_data)

AttributeError: 'dict' object has no attribute 'strftime'